## Dataset Preparation - Feature Creation and Data Preprocessing
### Kaggle Competition - Home Credit Default Risk - https://www.kaggle.com/c/home-credit-default-risk
#### Varun D N, varun.dn93@gmail.com

### Libraries

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### 1. Application_Train.csv

#### 1.1 Reading Data

In [4]:
# Training Data
application_train = pd.read_csv('application_train.csv')

In [3]:
# Test Data
application_test = pd.read_csv('application_test.csv')

In [5]:
# Sample rows
application_train.head(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
6,100009,0,Cash loans,F,Y,Y,1,171000.0,1560726.0,41301.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
7,100010,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,100011,0,Cash loans,F,N,Y,0,112500.0,1019610.0,33826.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
9,100012,0,Revolving loans,M,N,Y,0,135000.0,405000.0,20250.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
application_train.shape

(307511, 122)

There are about 300k loan samples in the training set with 122 attributes. These attributes can be used as features directly or might need some engineering to create derived features.

Custom Reference: 
    - Attributes are the columns given in a dataset
    - Features are the columns used for training
    - Attributes can be used directly as features and sometimes attributes can be used to create features

#### 1.2 Reviewing Missing Values and Imputation

In [46]:
# Counting missing values
num_missing_train = np.sum(application_train.isnull())
missing_in_train_cols = num_missing_train[num_missing_train != 0]
print missing_in_train_cols.sort_values()

EXT_SOURCE_2                       660
EMERGENCYSTATE_MODE             145755
TOTALAREA_MODE                  148431
YEARS_BEGINEXPLUATATION_MEDI    150007
YEARS_BEGINEXPLUATATION_AVG     150007
YEARS_BEGINEXPLUATATION_MODE    150007
FLOORSMAX_MEDI                  153020
FLOORSMAX_AVG                   153020
FLOORSMAX_MODE                  153020
HOUSETYPE_MODE                  154297
LIVINGAREA_MODE                 154350
LIVINGAREA_AVG                  154350
LIVINGAREA_MEDI                 154350
ENTRANCES_AVG                   154828
ENTRANCES_MODE                  154828
ENTRANCES_MEDI                  154828
APARTMENTS_MEDI                 156061
APARTMENTS_AVG                  156061
APARTMENTS_MODE                 156061
WALLSMATERIAL_MODE              156341
ELEVATORS_MODE                  163891
ELEVATORS_MEDI                  163891
ELEVATORS_AVG                   163891
NONLIVINGAREA_AVG               169682
NONLIVINGAREA_MEDI              169682
NONLIVINGAREA_MODE       

The methodology I am adopting to impute missing values is the following:
    - Identify the attributes with less than 25% of its values missing
    - If it's a categorical attribute, for example 'gender', impute the missing values with the mode of the values in the training set
    - If it's a continuous attribute, for example 'age', impute the missing values with either the mean or the median of the values in the training set depending upon the skew in the distribution

### Imputation

In [5]:
# Replacing the null values in AMT_ANNUITY with the median value in the dataset
application_train.loc[application_train.AMT_ANNUITY.isnull(), 'AMT_ANNUITY'] = \
    np.median(application_train[~ application_train.AMT_ANNUITY.isnull()].AMT_ANNUITY)

# Replacing the null values in AMT_GOODS_PRICE with the median value in the dataset
application_train.loc[application_train.AMT_GOODS_PRICE.isnull(), 'AMT_GOODS_PRICE'] = \
    np.median(application_train[~ application_train.AMT_GOODS_PRICE.isnull()].AMT_GOODS_PRICE)

# Replacing the null values in NAME_TYPE_SUITE with the median value in the dataset
application_train.loc[application_train.NAME_TYPE_SUITE.isnull(), 'NAME_TYPE_SUITE'] = 'Unaccompanied'

# Replacing the null values in OCCUPATION_TYPE with 'Unknown'. Not knowing the occupation can be a valuable feature.
application_train.loc[application_train.OCCUPATION_TYPE.isnull(), 'OCCUPATION_TYPE'] = 'Unknown'

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.CNT_FAM_MEMBERS.isnull(), 'CNT_FAM_MEMBERS'] = \
    np.mean(application_train[~ application_train.CNT_FAM_MEMBERS.isnull()].CNT_FAM_MEMBERS)

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.OBS_30_CNT_SOCIAL_CIRCLE.isnull(), 'OBS_30_CNT_SOCIAL_CIRCLE'] = \
    np.mean(application_train[~ application_train.OBS_30_CNT_SOCIAL_CIRCLE.isnull()].OBS_30_CNT_SOCIAL_CIRCLE)

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.DEF_30_CNT_SOCIAL_CIRCLE.isnull(), 'DEF_30_CNT_SOCIAL_CIRCLE'] = \
    np.mean(application_train[~ application_train.DEF_30_CNT_SOCIAL_CIRCLE.isnull()].DEF_30_CNT_SOCIAL_CIRCLE)

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.OBS_60_CNT_SOCIAL_CIRCLE.isnull(), 'OBS_60_CNT_SOCIAL_CIRCLE'] = \
    np.mean(application_train[~ application_train.OBS_60_CNT_SOCIAL_CIRCLE.isnull()].OBS_60_CNT_SOCIAL_CIRCLE)

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.DEF_60_CNT_SOCIAL_CIRCLE.isnull(), 'DEF_60_CNT_SOCIAL_CIRCLE'] = \
    np.mean(application_train[~ application_train.DEF_60_CNT_SOCIAL_CIRCLE.isnull()].DEF_60_CNT_SOCIAL_CIRCLE)

# Replacing the null values in CNT_FAM_MEMBERS with the median value in the dataset
application_train.loc[application_train.DAYS_LAST_PHONE_CHANGE.isnull(), 'DAYS_LAST_PHONE_CHANGE'] = \
    np.mean(application_train[~ application_train.DAYS_LAST_PHONE_CHANGE.isnull()].DAYS_LAST_PHONE_CHANGE)

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_MON with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_MON.isnull(), 'AMT_REQ_CREDIT_BUREAU_MON'] = 0

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_WEEK with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_WEEK.isnull(), 'AMT_REQ_CREDIT_BUREAU_WEEK'] = 0

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_DAY with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_DAY.isnull(), 'AMT_REQ_CREDIT_BUREAU_DAY'] = 0

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_HOUR with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_HOUR.isnull(), 'AMT_REQ_CREDIT_BUREAU_HOUR'] = 0

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_QRT with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_QRT.isnull(), 'AMT_REQ_CREDIT_BUREAU_QRT'] = 0

# Replacing the null values in AMT_REQ_CREDIT_BUREAU_YEAR with 0
application_train.loc[application_train.AMT_REQ_CREDIT_BUREAU_YEAR.isnull(), 'AMT_REQ_CREDIT_BUREAU_YEAR'] = 0

# Replacing the null values in EXT_SOURCE_3 with mean 
application_train.loc[application_train.EXT_SOURCE_3.isnull(), 'EXT_SOURCE_3'] = \
    np.mean(application_train[~ application_train.EXT_SOURCE_3.isnull()].EXT_SOURCE_3)

# Replacing the null values in EXT_SOURCE_2 with mean 
application_train.loc[application_train.EXT_SOURCE_2.isnull(), 'EXT_SOURCE_2'] = \
    np.mean(application_train[~ application_train.EXT_SOURCE_2.isnull()].EXT_SOURCE_2)

#### 1.2.1 Filtering Attributes Without Any Missing Values

In [48]:
# Count of all missing values
num_missing_values = np.sum(application_train.isnull())

# Filter columns with no missing values
no_missing_values_columns = num_missing_values[num_missing_values == 0].index.values

#### 1.3 Creating the Training Dataset With Complete Attributes

In [50]:
application_train_selected_feats = application_train[no_missing_values_columns]

application_train_selected_feats.shape

(307511, 73)

#### 1.4 Creating Indicator Features of Categorical Attributes
###### In this process below, dummy variables are created for each categorical attribute, for example NAME_CONTRACT_TYPE, NAME_INCOME_TYPE. The different values within each attribute will be listed and each value will be created as a new indicator variable (dummy variable) and the dummy variable will be filled with 1 or 0 depending upon the presence or absence of it for a loan sample. 

In [51]:
# Creating dummy variables for categorical features
application_train_selected_feats = pd.get_dummies(application_train_selected_feats)

#### 1.5 Verification of Absence of Missing Values

In [52]:
np.sum(np.sum(application_train_selected_feats.isnull()))

0

### 2. Bureau.csv

#### 2.1. Reading Data

In [3]:
bureau = pd.read_csv('bureau.csv')

In [4]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


#### 2.2. Feature Creation for Loan Samples 

In [13]:
# Number of applications
num_applications_cb = bureau.groupby('SK_ID_CURR').size().reset_index()
num_applications_cb.columns = ['SK_ID_CURR', 'num_applications_cb']

# Maximum credit enddate days
max_days_credit_enddate = \
    bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].max().reset_index()
max_days_credit_enddate.columns = ['SK_ID_CURR', 'max_days_credit_enddate']

# Latest credit update
latest_credit_update = \
    bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].max().reset_index()
latest_credit_update.columns = ['SK_ID_CURR', 'latest_credit_update']

# Current debt
current_debt_cb = bureau[bureau.CREDIT_ACTIVE == 'Active'].groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].max().reset_index()
current_debt_cb.columns = ['SK_ID_CURR', 'current_debt_cb']

# Count different credit types
count_credit_types = pd.DataFrame(bureau.groupby('SK_ID_CURR')['CREDIT_TYPE'].value_counts())
count_credit_types.columns = ['count_credit_type']
count_credit_types = count_credit_types.reset_index()

# Count consumer credit
count_consumer_credit = \
    count_credit_types[count_credit_types.CREDIT_TYPE == 'Consumer credit'][['SK_ID_CURR', 'count_credit_type']]
count_consumer_credit.columns = ['SK_ID_CURR', 'count_consumer_credit']

# Count credit cards
count_credit_card = count_credit_types[count_credit_types.CREDIT_TYPE == 'Credit card'][['SK_ID_CURR', 'count_credit_type']]
count_credit_card.columns = ['SK_ID_CURR', 'count_credit_card']

# Count any other credit types
count_other_credit_types = \
    count_credit_types[(count_credit_types.CREDIT_TYPE != 'Credit card') & \
                       (count_credit_types.CREDIT_TYPE != 'Consumer credit')]\
                      [['SK_ID_CURR', 'count_credit_type']]
count_other_credit_types.columns = ['SK_ID_CURR', 'count_other_credit_types']

# Number of credit currencies
num_currency = bureau.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].nunique().reset_index()

# Total days overdue
total_cb_days_overdue = bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].sum().reset_index()

# Total number of prolonged credit
total_cnt_credit_prolong = bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum().reset_index()

# Total credit sum - overdue
total_credit_sum_overdue = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index()

In [94]:
application_train_selected_feats.head(10)

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,num_applications_cb,max_days_credit_enddate,latest_credit_update,current_debt_cb,count_consumer_credit,count_credit_card,count_other_credit_types
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,8.0,780.0,-7.0,245781.0,4.0,4.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,4.0,1216.0,-43.0,0.0,2.0,2.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,100008,0,0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941,-1588,...,0,0,0,3.0,471.0,-16.0,240057.0,3.0,0.0,0.0
6,100009,0,1,171000.0,1560726.0,41301.0,1395000.0,0.035792,-13778,-3130,...,0,0,0,18.0,1402.0,-23.0,557959.5,16.0,2.0,0.0
7,100010,0,0,360000.0,1530000.0,42075.0,1530000.0,0.003122,-18850,-449,...,0,0,0,2.0,689.0,-18.0,348007.5,1.0,0.0,1.0
8,100011,0,0,112500.0,1019610.0,33826.5,913500.0,0.018634,-20099,365243,...,0,0,1,4.0,0.0,0.0,0.0,3.0,1.0,0.0
9,100012,0,0,135000.0,405000.0,20250.0,405000.0,0.019689,-14469,-2019,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3. Previous Application.csv

#### 3.1. Reading Data

In [2]:
previous_application = pd.read_csv('previous_application.csv')

In [3]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.2. Feature Creation for Loan Samples

In [5]:
# Difference between requested and approved credit
previous_application['diff_credit_requested_approved'] = previous_application.AMT_APPLICATION - previous_application.AMT_CREDIT

In [6]:
# Number of previous contract types
num_contract_types_prev = previous_application.groupby('SK_ID_CURR').size().reset_index()
num_contract_types_prev.columns = ['SK_ID_CURR', 'num_contract_types_prev']

# Past average annuity 
avg_annuity_prev = previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].mean().reset_index()
avg_annuity_prev.columns = ['SK_ID_CURR', 'avg_annuity_prev']

# Maximum credit requested in the past
max_credit_requested_prev = previous_application.groupby('SK_ID_CURR')['AMT_APPLICATION'].max().reset_index()
max_credit_requested_prev.columns = ['SK_ID_CURR', 'max_credit_requested_prev']

# Maximum credit approved in the past
max_credit_approved_prev = previous_application.groupby('SK_ID_CURR')['AMT_CREDIT'].max().reset_index()
max_credit_approved_prev.columns = ['SK_ID_CURR', 'max_credit_approved_prev']

# Sum of difference in the credit requested and approved in the past
total_diff_credit_requested_approved = previous_application.groupby('SK_ID_CURR')['diff_credit_requested_approved'].sum().reset_index()
total_diff_credit_requested_approved.columns = ['SK_ID_CURR', 'total_diff_credit_requested_approved']

# Maximum rate of downpayment in the past
max_rate_down_payment_prev = previous_application.groupby('SK_ID_CURR')['RATE_DOWN_PAYMENT'].max().reset_index()
max_rate_down_payment_prev.columns = ['SK_ID_CURR', 'max_rate_down_payment_prev']

# Maximum primary interest rate in the past
max_rate_primary_prev = previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIMARY'].max().reset_index()
max_rate_primary_prev.columns = ['SK_ID_CURR', 'max_rate_primary_prev']

# Maximum privileged interest rate
max_rate_privileged_prev = previous_application.groupby('SK_ID_CURR')['RATE_INTEREST_PRIVILEGED'].max().reset_index()
max_rate_privileged_prev.columns = ['SK_ID_CURR', 'max_rate_privileged_prev']

# Total contracts in the past
total_contracts_prev = previous_application.groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].size().reset_index()
total_contracts_prev.columns = ['SK_ID_CURR', 'total_contracts']

# Total approved contracts in the past
total_approved_contracts_prev = previous_application[previous_application.NAME_CONTRACT_STATUS == 'Approved'].groupby('SK_ID_CURR').size().reset_index()
total_approved_contracts_prev.columns = ['SK_ID_CURR', 'total_approved_contracts']

# Ratio of approved and requested contracts
temp = pd.merge(total_contracts_prev, total_approved_contracts_prev, on = 'SK_ID_CURR', how = 'left')
temp = temp.fillna(0)
temp['perc_approved_contracts_prev'] = temp.total_approved_contracts / temp.total_contracts

In [12]:
application_train_selected_feats.shape

(310078, 223)

### 4. POS CASH balance.csv

#### 4.1. Reading Data

In [11]:
pos_cash_balance = pd.read_csv('POS_CASH_balance.csv')

In [125]:
pos_cash_balance.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
5,2207092,342166,-32,12.0,12.0,Active,0,0
6,1110516,204376,-38,48.0,43.0,Active,0,0
7,1387235,153211,-35,36.0,36.0,Active,0,0
8,1220500,112740,-31,12.0,12.0,Active,0,0
9,2371489,274851,-32,24.0,16.0,Active,0,0


#### 4.2. Feature Creation for POS CASH Balance

In [18]:
# Total DPD
total_dpd_overall = pos_cash_balance.groupby('SK_ID_CURR')['SK_DPD'].sum().reset_index()
total_dpd_overall.columns = ['SK_ID_CURR', 'total_dpd_overall']

# Ratio of finished installements
pos_cash_balance['perc_finished_installments'] = \
    (pos_cash_balance.CNT_INSTALMENT - pos_cash_balance.CNT_INSTALMENT_FUTURE) / pos_cash_balance.CNT_INSTALMENT

# Average of the ratio of finished installments
avg_perc_finished_installments = pos_cash_balance.groupby('SK_ID_CURR')['perc_finished_installments'].mean().reset_index()

### 5. Credit Card Balance.csv

#### 5.1. Reading Data

In [4]:
credit_card_balance = pd.read_csv('credit_card_balance.csv')

In [5]:
credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


#### 5.2. Feature Creation for Credit Card Balance

In [13]:
# Average cash withdrawl in the ATMs
average_atm_cash_drawing = credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_ATM_CURRENT'].mean().reset_index()

# Average current drawings
average_current_drawing = credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].mean().reset_index()

# Average balance amount
average_amt_balance = credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].mean().reset_index()

# Maximum cash withdrawn in the ATMs
max_atm_cash_drawing = credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_ATM_CURRENT'].max().reset_index()

# Maximum current withdrawl
max_current_drawing = credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].max().reset_index()

# Maximum amount in the balance
max_amt_balance = credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].max().reset_index()

# Total amount withdrawn from the ATM
total_atm_drawing = credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_ATM_CURRENT'].sum().reset_index()

# Total number of current withdrawls
total_current_drawing = credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].sum().reset_index()

# Total POS current withdrawls
total_pos_current_drawing = credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_POS_CURRENT'].sum().reset_index()

# Total other types of withdrawls
total_other_current_drawing = credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_OTHER_CURRENT'].sum().reset_index()

# Total paid installments
total_paid_installments = credit_card_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_MATURE_CUM'].sum().reset_index()

### 6. Installments Payments.csv

#### 6.1. Reading Data

In [99]:
installments_payments = pd.read_csv('installments_payments.csv')

In [100]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


#### 6.2. Feature Creation for Installments Payments

In [104]:
# Number of days for which installment was delayed
installments_payments['days_installment_delayed'] = \
    installments_payments.DAYS_INSTALMENT - installments_payments.DAYS_ENTRY_PAYMENT
    
# Difference in the installment amount and paid amount
installments_payments['amt_payment_diff'] = \
    installments_payments.AMT_INSTALMENT - installments_payments.AMT_PAYMENT

# Total number of delayed installment days
delayed_installments_payments = installments_payments.groupby('SK_ID_CURR')['days_installment_delayed'].sum().reset_index()
delayed_installments_payments.columns = ['SK_ID_CURR', 'sum_days_delayed']

# Sum of differences in installment payments
amt_payment_diff = installments_payments.groupby('SK_ID_CURR')['amt_payment_diff'].sum().reset_index()
amt_payment_diff.columns = ['SK_ID_CURR', 'sum_lesser_payments']

### 7. Merging the Features from Other Datasets Into the Application Train Dataset

In [14]:
# Credit Bureau
application_train_selected_feats = pd.merge(application_train_selected_feats, num_applications_cb, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_days_credit_enddate, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, latest_credit_update, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, current_debt_cb, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, count_consumer_credit, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, count_credit_card, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, count_other_credit_types, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_credit_sum_overdue, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_cnt_credit_prolong, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_cb_days_overdue, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, num_currency, on = 'SK_ID_CURR', how = 'left')

# Previous Application
application_train_selected_feats = pd.merge(application_train_selected_feats, num_contract_types_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, avg_annuity_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_credit_requested_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_credit_approved_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_diff_credit_requested_approved, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_rate_down_payment_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_rate_primary_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_rate_privileged_prev, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, temp, on = 'SK_ID_CURR', how = 'left')

# POS CASH Balance 
application_train_selected_feats = pd.merge(application_train_selected_feats, total_dpd_overall, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, avg_perc_finished_installments, on = 'SK_ID_CURR', how = 'left')

# Installments Payments
application_train_selected_feats = pd.merge(application_train_selected_feats, delayed_installments_payments, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, amt_payment_diff, on = 'SK_ID_CURR', how = 'left')

# Credit Card Balance
application_train_selected_feats = pd.merge(application_train_selected_feats, average_atm_cash_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, average_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, average_amt_balance, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_atm_cash_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, max_amt_balance, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_atm_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_pos_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_other_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_train_selected_feats = pd.merge(application_train_selected_feats, total_paid_installments, on = 'SK_ID_CURR', how = 'left')

# Imputing all the missing values in these continuous features with 0
application_train_selected_feats = application_train_selected_feats.fillna(0)

#### Saving the Final Training Data

In [14]:
application_train_selected_feats.to_csv('application_train_selected_feats.csv', index = False)

In [13]:
application_train_selected_feats.shape

(310078, 223)

### 8. Test Data Preparation

Replicating the feature creation work done for training data

In [84]:
application_test_selected_feats = application_test[[c for c in no_missing_values_columns if c != 'TARGET']]

In [85]:
application_test_selected_feats.shape

(48744, 72)

#### 8.1. Missing Values and Imputation

In [86]:
# Imputing missing values for features as per the strategy adopted for features in the training data
test_data_missing_values = np.sum(application_test.isnull())
cols_with_missing_values = test_data_missing_values[test_data_missing_values != 0].index

In [87]:
# Categorical Types
categorical_missing_feats = [c for c in application_test_selected_feats.select_dtypes(include=['object']) if c in cols_with_missing_values]

for c in categorical_missing_feats:
    application_test_selected_feats.loc[application_test_selected_feats[c].isnull(), c] = application_train[c].value_counts().idxmax()

In [88]:
# Numerical Types
numerical_missing_feats = [c for c in application_test_selected_feats.select_dtypes(include=['int64', 'float64', 'uint8']) if c in cols_with_missing_values]

for c in numerical_missing_feats:
    print c
    if 'AMT_REQ' in c:
        application_test_selected_feats.loc[application_test_selected_feats[c].isnull(), c] = 0
    else:
        application_test_selected_feats.loc[application_test_selected_feats[c].isnull(), c] = np.median(application_train[~ application_train[c].isnull()][c])

AMT_ANNUITY
EXT_SOURCE_2
EXT_SOURCE_3
OBS_30_CNT_SOCIAL_CIRCLE
DEF_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE
AMT_REQ_CREDIT_BUREAU_HOUR
AMT_REQ_CREDIT_BUREAU_DAY
AMT_REQ_CREDIT_BUREAU_WEEK
AMT_REQ_CREDIT_BUREAU_MON
AMT_REQ_CREDIT_BUREAU_QRT
AMT_REQ_CREDIT_BUREAU_YEAR


In [89]:
application_test_selected_feats.shape

(48744, 72)

In [ ]:
# Creating Dummy Variables
application_test_selected_feats = pd.get_dummies(application_test_selected_feats)

#### 8.2. Merging Features from Other Datasets

In [18]:
application_test_selected_feats = pd.merge(application_test_selected_feats, num_applications_cb, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_days_credit_enddate, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, latest_credit_update, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, current_debt_cb, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = pd.merge(application_test_selected_feats, count_consumer_credit, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, count_credit_card, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, count_other_credit_types, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_dpd_overall, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = pd.merge(application_test_selected_feats, delayed_installments_payments, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, amt_payment_diff, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = pd.merge(application_test_selected_feats, average_atm_cash_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, average_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, average_amt_balance, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_atm_cash_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_amt_balance, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_atm_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_pos_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_other_current_drawing, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_paid_installments, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = pd.merge(application_test_selected_feats, avg_perc_finished_installments, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_credit_sum_overdue, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_cnt_credit_prolong, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_cb_days_overdue, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, num_currency, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = pd.merge(application_test_selected_feats, num_contract_types_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, avg_annuity_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_credit_requested_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_credit_approved_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, total_diff_credit_requested_approved, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_rate_down_payment_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_rate_primary_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, max_rate_privileged_prev, on = 'SK_ID_CURR', how = 'left')
application_test_selected_feats = pd.merge(application_test_selected_feats, temp, on = 'SK_ID_CURR', how = 'left')

application_test_selected_feats = application_test_selected_feats.fillna(0)

In [109]:
application_test_selected_feats.shape

(48744, 195)

In [110]:
# Removing any new columns in test set not present in training set
for c in application_train_selected_feats.columns:
    if c not in application_test_selected_feats.columns:
        application_test_selected_feats[c] = [0] * application_test_selected_feats.shape[0]

In [111]:
application_test_selected_feats = application_test_selected_feats[application_train_selected_feats.columns]
application_test_selected_feats = application_test_selected_feats.drop('TARGET', axis = 1)

In [112]:
application_test_selected_feats.shape

(48744, 195)

In [113]:
application_test_selected_feats = application_test_selected_feats.drop_duplicates(subset = ['SK_ID_CURR'])

###### Making sure the training set has 1 more column than the test set. The missing column in the test set has to be the "TARGET" attribute.

In [18]:
print application_train_selected_feats.shape, application_test_selected_feats.shape

(310078, 223) (48744, 222)


####  Saving Final Test Dataset

In [19]:
application_test_selected_feats.to_csv('application_test_selected_feats.csv', index = False)